In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataset = pd.read_csv("Users/sa12/Documents/Repositories/WeatherDataMG/data/csv/measurements.csv")

In [ ]:
dataset.describe()

In [ ]:
dataset.plot.hist()

In [ ]:
monthly = dataset.resample("1M")
monthly.describe()

In [ ]:
yearly = dataset.resample("1Y")
yearly.describe()